# Convert OPL dat file to JSON

## Uses pyparsing

## Scroll down to input your filename

This will not handle some OPL statements, and you will need to clear the .dat file of these statements before converting to json:
* prepare { ... }
* DBConnection, read from DB, write to DB
* SheetConnection, read from xls, write to xls

In [1]:
from __future__ import print_function
import pyparsing as pp

In [2]:
# general parsing expressions

open_bracket = pp.Literal('[')
close_bracket = pp.Literal(']')
open_brace = pp.Literal('{')
close_brace = pp.Literal('}')
open_angle = pp.Literal('<')
close_angle = pp.Literal('>')

zero = pp.Literal('0')

def pr_unsigned_int(toks):     # remove leading 0's; JSON does not support leading 0's!
    if len(toks[0]) > 1:
        while toks[0][0] == '0':
            toks[0] = toks[0][1:]
    return toks

unsigned_int = pp.Word(pp.nums)
unsigned_int.setParseAction(pr_unsigned_int)

integer = pp.Combine( pp.Optional(pp.oneOf("+ -")) + unsigned_int)
real = pp.Combine( pp.Optional(pp.oneOf("+ -")) + pp.Word(pp.nums) + "." + pp.Optional(pp.Word(pp.nums)) )
number =  real ^ integer      # order is important!

identifier = pp.Word( pp.alphas, pp.alphanums+"_" )
token = pp.Word( pp.alphanums+"_[]-()", pp.alphanums+"_[]-()" )
atom = integer | identifier

field_item = number | pp.dblQuotedString | identifier

end_stmnt = pp.Literal(';').suppress()

testStrings = [ 
 '56',
 '0',
 '+34',
 '-007',
 '45.67',
 '0.5',
 '05',
 'this',
 '"that"']

for testStr in testStrings:
    assert(field_item.parseString(testStr))
    print(field_item.parseString(testStr))

['56']
['0']
['+34']
['-7']
['45.67']
['0.5']
['5']
['this']
['"that"']


In [3]:
# parse arrays (add commas if not already present)
# output is json style arrays
testStrings = [ 
 '[ 0.001,  0.002 ]',
 '[ 0     50000 30000 50000 ]',
 '[ 35, 15, 40, 15, 05, 10, 05, 10, 05, 05 ]',
 '[[0,1], [1,0]]',
 '[[0 1] [1 0]]',
 '[[0, 1] [1, 0]]',
 '[[0 1], [1 0]]',
     '  < "L", 1, 2, 1, 10000 >',
     '  < "L"  1  2  1  10000 >',
    """
    {
   < "L", 1, 2, 1, 10000 >
   < "U", 2, 2, 2, 10000 >
       }
""",
    """
{<1,1,3,1400>,     <1,2,5,800>,     <1,3,1,2000>, 
     <2,1,6,1250>, 
     <3,1,3,1000>,     <3,2,1,500>,     <3,3,4,300>, 
     <4,1,1,800>,     <4,2,4,1600>, 
     <5,1,5,900>, 
     <6,1,5,2250>,     <6,2,6,1125>,     <6,3,2,1800>, 
     <7,1,5,2000>,     <7,2,2,1000> 
     }
"""              
]

def pr_array(toks):
    return '[ ' + ', '.join(toks) + ' ]'

open_list = open_brace | open_bracket | open_angle
close_list = close_brace | close_bracket | close_angle

opl_array = pp.Forward()
opl_array << open_list.suppress() + (pp.OneOrMore(opl_array) ^ pp.delimitedList(opl_array) 
                                   ^ pp.OneOrMore(field_item) ^ pp.delimitedList(field_item)) + \
            close_list.suppress()
opl_array.setParseAction(pr_array)

for testStr in testStrings:
    assert(opl_array.parseString(testStr))
    print(opl_array.parseString(testStr))

['[ 0.001, 0.002 ]']
['[ 0, 50000, 30000, 50000 ]']
['[ 35, 15, 40, 15, 5, 10, 5, 10, 5, 5 ]']
['[ [ 0, 1 ], [ 1, 0 ] ]']
['[ [ 0, 1 ], [ 1, 0 ] ]']
['[ [ 0, 1 ], [ 1, 0 ] ]']
['[ [ 0, 1 ], [ 1, 0 ] ]']
['[ "L", 1, 2, 1, 10000 ]']
['[ "L", 1, 2, 1, 10000 ]']
['[ [ "L", 1, 2, 1, 10000 ], [ "U", 2, 2, 2, 10000 ] ]']
['[ [ 1, 1, 3, 1400 ], [ 1, 2, 5, 800 ], [ 1, 3, 1, 2000 ], [ 2, 1, 6, 1250 ], [ 3, 1, 3, 1000 ], [ 3, 2, 1, 500 ], [ 3, 3, 4, 300 ], [ 4, 1, 1, 800 ], [ 4, 2, 4, 1600 ], [ 5, 1, 5, 900 ], [ 6, 1, 5, 2250 ], [ 6, 2, 6, 1125 ], [ 6, 3, 2, 1800 ], [ 7, 1, 5, 2000 ], [ 7, 2, 2, 1000 ] ]']


In [4]:
# parse // comments
# all comments ignored, json does not support comments
testStrings = [ 
 '// Factory 1',
 '// --------------------------------------------------------------------------']

#dslash_lit = pp.Literal('//')
#opl_comment = dslash_lit + pp.restOfLine

def pr_comment(tok):
    return '{}\n'.format(tok[0].replace('//','#'))

opl_comment = pp.dblSlashComment | pp.cStyleComment
opl_comment.setParseAction(pr_comment)

for testStr in testStrings:
    assert(opl_comment.parseString(testStr))
    print(opl_comment.parseString(testStr))

['# Factory 1\n']
['# --------------------------------------------------------------------------\n']


In [5]:
# parse keyed arrays #[ key : value ]#  =>  { key : value }
# output json style dictionaries
testStrings = [ """
    #[
        1: [ 0.001  0.002 ]   // Factory 1
        2: [ 0.002  0.003 ]   // Factory 2
    ]#
""",
"""
    #[ 
      2: #[ // Stage 2
         1: [ 20000 30000 15000 40000 ]  // Product 1
         2: [ 0     50000 30000 50000 ]  // Product 2
          ]#
      3: #[ // Stage 3
         1: [ 10000 5000  15000 40000 ]  // Product 1
         2: [ 0     10000     0  5000 ]  // Product 2
          ]#
    ]#
"""              
]

dict_beg = pp.Literal('#[') | pp.Literal('#<')
dict_end = pp.Literal(']#') | pp.Literal('>#')
dict_key = field_item
colon_lit = pp.Literal(':')
opl_dict = pp.Forward()
dict_value = opl_array | field_item | opl_dict

def pr_entry(toks):
    if toks[0][0] == '"':
        return " ".join(toks)
    else:
        return '"' + toks[0] + '"' + " ".join(toks[1:])

dict_entry = dict_key + colon_lit + (pp.Optional(opl_comment)).suppress() + dict_value + \
                                    (pp.Optional(opl_comment)).suppress()
dict_entry.setParseAction(pr_entry)

def pr_dict(toks):
    return  '{\n' + ',\n'.join(toks) + '\n}'

opl_dict << dict_beg.suppress() + (pp.Optional(opl_comment)).suppress() + \
                                    (pp.OneOrMore(dict_entry) ^ pp.delimitedList(dict_entry)) + \
                                 (pp.Optional(opl_comment)).suppress() + \
            dict_end.suppress()
opl_dict.setParseAction(pr_dict)

for testStr in testStrings:
    assert(opl_dict.parseString(testStr))
    print(opl_dict.parseString(testStr))

['{\n"1": [ 0.001, 0.002 ],\n"2": [ 0.002, 0.003 ]\n}']
['{\n"2": {\n"1": [ 20000, 30000, 15000, 40000 ],\n"2": [ 0, 50000, 30000, 50000 ]\n},\n"3": {\n"1": [ 10000, 5000, 15000, 40000 ],\n"2": [ 0, 10000, 0, 5000 ]\n}\n}']


In [6]:
def pr_stmnt(toks=None):
    if toks:
        #print(toks)
        return ' '.join(toks[0])
    else:
        return None

opl_expr = number | opl_array | opl_dict 

def add_quotes(tok):
    return '"' + tok[0] + '"'

equal_lit = pp.Literal('=').setParseAction(pp.replaceWith(":"))
opl_assign = pp.Group(identifier.setParseAction(add_quotes) + equal_lit + opl_expr + end_stmnt)
opl_stmnt = opl_assign | opl_comment.suppress()
opl_stmnt.setParseAction(pr_stmnt)

def pr_opldat(toks):
    return '{\n' + ',\n\n'.join(toks) + '\n}'

opldat = pp.OneOrMore(opl_stmnt)
opldat.setParseAction(pr_opldat)

{{Group:({{{W:(abcd...,abcd...) "="} {{{Combine:({[Re:('[+\\-]')] W:(0123...) "." [W:(0123...)]}) ^ Combine:({[Re:('[+\\-]')] W:(0123...)})} | Forward: {Suppress:({"{" | "[" | "<"}) {{...}... ^ Forward: None [, Forward: None]... ^ {{{Combine:({[Re:('[+\\-]')] W:(0123...) "." [W:(0123...)]}) ^ Combine:({[Re:('[+\\-]')] W:(0123...)})} | string enclosed in double quotes | W:(abcd...,abcd...)}}... ^ {{Combine:({[Re:('[+\\-]')] W:(0123...) "." [W:(0123...)]}) ^ Combine:({[Re:('[+\\-]')] W:(0123...)})} | string enclosed in double quotes | W:(abcd...,abcd...)} [, {{Combine:({[Re:('[+\\-]')] W:(0123...) "." [W:(0123...)]}) ^ Combine:({[Re:('[+\\-]')] W:(0123...)})} | string enclosed in double quotes | W:(abcd...,abcd...)}]...} Suppress:({"}" | "]" | ">"})}} | Forward: {Suppress:({"#[" | "#<"}) Suppress:([{// comment | C style comment}]) {{{{{Combine:({[Re:('[+\\-]')] W:(0123...) "." [W:(0123...)]}) ^ Combine:({[Re:('[+\\-]')] W:(0123...)})} | string enclosed in double quotes | W:(abcd...,abcd.

# To convert an OPL dat file to a JSON file,  just fill in the filename below and run this cell

## JSON will be printed as cell output and saved in a file with json extension

In [7]:
filename = 'supplydata.dat'
#filename = join(join(example_dir,'fixed'),'fixed.dat'))

po = opldat.parseFile(filename)

if filename[-3:] == 'dat':
    jsonfile = filename[:-3] + 'json'
else:
    jsonfile = filename + 'json'
    
with open(jsonfile, 'w') as g:
    for item in po:
        print(item)
        g.write(item)

{
"RequiredLotSize" : 0,

"Demand" : {
"2": {
"1": [ 20000, 30000, 15000, 40000 ],
"2": [ 0, 50000, 30000, 50000 ]
},
"3": {
"1": [ 10000, 5000, 15000, 40000 ],
"2": [ 0, 10000, 0, 5000 ]
}
},

"ProdCost" : {
"1": [ 1, 0.5 ],
"2": [ 0.5, 0.25 ]
},

"ProdTime" : {
"1": [ 0.001, 0.002 ],
"2": [ 0.002, 0.003 ]
},

"HoldCost" : 0.1,

"MovCost_FDC" : [ 0.1, 0.3 ],

"MovCost_FC" : [ 0.05, 0.05 ],

"MovCost_DCC" : 0.5,

"TardCost_DC" : [ 10, 5 ],

"TardCost_C" : [ 20, 15 ],

"Penalty" : 1000,

"bnds" : [ [ "L", 1, 2, 1, 10000 ], [ "U", 2, 2, 2, 10000 ] ]
}
